<a href="https://colab.research.google.com/github/mmetawei/AFQC/blob/main/Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 58.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from qiskit import transpile
from qiskit.circuit.library import efficient_su2  # ← FIXED: lowercase function
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error, pauli_error
from qiskit.quantum_info import SparsePauliOp, Statevector
import time


qubits_list = [2, 3, 4, 5]
layers_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
error_rates = [0.01, 0.02, 0.03, 0.05, 0.08, 0.10]
noise_types = ['depolarizing', 'bit_flip', 'phase_flip']
num_circuits_per_config = 200


def create_ansatz(num_qubits, reps):

    return efficient_su2(num_qubits=num_qubits, reps=reps, entanglement='linear')

def create_observable(num_qubits):
    """Create ZZ observable on neighboring qubits."""
    terms = []
    for i in range(num_qubits - 1):
        pauli_str = 'I' * i + 'ZZ' + 'I' * (num_qubits - i - 2)
        terms.append((pauli_str, 1.0))
    return SparsePauliOp.from_list(terms)

def create_noise_model(noise_type, error_1q, error_2q):
    """
    Create a realistic noise model that covers all common
    1Q and 2Q gates generated by efficient_su2 + transpilation.
    """

    noise_model = NoiseModel()

    # -------------------------------
    # Define error channels
    # -------------------------------
    if noise_type == 'depolarizing':
        error_1q_gate = depolarizing_error(error_1q, 1)
        error_2q_gate = depolarizing_error(error_2q, 2)

    elif noise_type == 'bit_flip':
        error_1q_gate = pauli_error([
            ('X', error_1q),
            ('I', 1 - error_1q)
        ])
        error_2q_gate = pauli_error([
            ('XX', error_2q / 3),
            ('XI', error_2q / 3),
            ('IX', error_2q / 3),
            ('II', 1 - error_2q)
        ])

    elif noise_type == 'phase_flip':
        error_1q_gate = pauli_error([
            ('Z', error_1q),
            ('I', 1 - error_1q)
        ])
        error_2q_gate = pauli_error([
            ('ZZ', error_2q / 3),
            ('ZI', error_2q / 3),
            ('IZ', error_2q / 3),
            ('II', 1 - error_2q)
        ])

    else:
        raise ValueError(f"Unknown noise type: {noise_type}")

    # -------------------------------
    # 1-qubit gates to noise
    # -------------------------------
    one_qubit_gates = [
        'rx', 'ry', 'rz',      # rotation gates
        'h', 's', 'sdg',       # Clifford gates
        'u', 'u1', 'u2', 'u3'  # generic single-qubit gates
    ]

    for gate in one_qubit_gates:
        noise_model.add_all_qubit_quantum_error(error_1q_gate, gate)

    # -------------------------------
    # 2-qubit gates to noise
    # -------------------------------
    two_qubit_gates = ['cx', 'cz', 'ecr']

    for gate in two_qubit_gates:
        noise_model.add_all_qubit_quantum_error(error_2q_gate, gate)

    return noise_model

#def create_noise_model(noise_type, error_1q, error_2q):
    #"""Create noise model based on noise type."""
    #noise_model = NoiseModel()

    #if noise_type == 'depolarizing':
      #  error_1q_gate = depolarizing_error(error_1q, 1)
     #   error_2q_gate = depolarizing_error(error_2q, 2)

    #elif noise_type == 'bit_flip':
        #error_1q_gate = pauli_error([('X', error_1q), ('I', 1 - error_1q)])
        #error_2q_gate = pauli_error([
         #   ('XX', error_2q/3),
        #    ('XI', error_2q/3),
       #     ('IX', error_2q/3),
      #      ('II', 1 - error_2q)
     #   ])

    #elif noise_type == 'phase_flip':
       # error_1q_gate = pauli_error([('Z', error_1q), ('I', 1 - error_1q)])
       # error_2q_gate = pauli_error([
       #     ('ZZ', error_2q/3),
      #      ('ZI', error_2q/3),
     #       ('IZ', error_2q/3),
    #        ('II', 1 - error_2q)
   #     ])

  #  noise_model.add_all_qubit_quantum_error(error_1q_gate, ['ry', 'rz'])
 #   noise_model.add_all_qubit_quantum_error(error_2q_gate, ['cx'])

#    return noise_model

def get_ideal_expectation(circuit, observable):
    sv = Statevector.from_instruction(circuit)
    return float(np.real(sv.expectation_value(observable)))

def get_noisy_expectation(circuit, noisy_sim, observable, shots=2048):
    """
    Compute ⟨observable⟩ under noise using correct Pauli-basis measurements.

    Parameters
    ----------
    circuit : QuantumCircuit
        Parameter-bound quantum circuit (no measurements).
    noisy_sim : AerSimulator
        Simulator with noise model attached.
    observable : SparsePauliOp
        Observable to measure.
    shots : int
        Number of shots per Pauli term.

    Returns
    -------
    float
        Estimated expectation value.
    """

    total_expectation = 0.0

    # Loop over Pauli terms (e.g., 'ZZII', ...)
    for pauli_str, coeff in observable.to_list():

        qc = circuit.decompose().copy()

        # --- Basis change for each qubit ---
        for q, p in enumerate(reversed(pauli_str)):
            if p == 'X':
                qc.h(q)
            elif p == 'Y':
                qc.sdg(q)
                qc.h(q)
            # Z and I → no basis change

        qc.measure_all()

        # Transpile & run
        basis = ['rx', 'ry', 'rz', 'cx']
        transpiled = transpile(qc, backend=noisy_sim, basis_gates=basis)
        job = noisy_sim.run(transpiled, shots=shots)
        counts = job.result().get_counts()

        # --- Compute expectation for this Pauli term ---
        exp_val = 0.0
        for bitstring, count in counts.items():
            bitstring = bitstring.replace(" ", "")
            parity = 1

            for q, p in enumerate(reversed(pauli_str)):
                if p != 'I' and bitstring[q] == '1':
                    parity *= -1

            exp_val += parity * count

        exp_val /= shots
        total_expectation += coeff * exp_val

    return float(np.real(total_expectation))

#def get_noisy_expectation(circuit, noisy_sim, observable, num_qubits, shots=4096):
    #decomposed = circuit.decompose()
    #meas_circuit = decomposed.copy()
    #meas_circuit.measure_all()

    #transpiled = transpile(meas_circuit, backend=noisy_sim)
    #job = noisy_sim.run(transpiled, shots=shots)
    #counts = job.result().get_counts()

    #exp_val = 0
    #for pauli_str, coeff in observable.to_list():
        #pauli_exp = 0
        #for bitstring, count in counts.items():
           # bitstring = bitstring.replace(" ", "")
          #  parity = 1
         #   for i, p in enumerate(reversed(pauli_str)):
        #        if p == 'Z':
       #             if bitstring[-(i+1)] == '1':
      #                  parity *= -1
     #       pauli_exp += parity * count
    #    pauli_exp /= shots
   #     exp_val += coeff * pauli_exp

  #  return exp_val.real


print("="*70)
print("QEM DATABASE GENERATOR (FIXED)")
print("="*70)

total_circuits = len(qubits_list) * len(layers_list) * num_circuits_per_config
print(f"Total circuits: {total_circuits:,}")


simulators = {}
for noise_type in noise_types:
    simulators[noise_type] = {}
    for i, err in enumerate(error_rates):
        error_2q = err * 2
        sim = AerSimulator(noise_model=create_noise_model(noise_type, err, error_2q))
        simulators[noise_type][i] = sim


data = []
np.random.seed(42)
current = 0
start_time = time.time()

for num_qubits in qubits_list:
    observable = create_observable(num_qubits)
    print(f"\nProcessing {num_qubits} qubits...")

    for reps in layers_list:
        ansatz = create_ansatz(num_qubits, reps)
        num_params = ansatz.num_parameters

        for circuit_idx in range(num_circuits_per_config):
            params = np.random.uniform(0, 2*np.pi, num_params)
            bound_circuit = ansatz.assign_parameters(params)

            x_ideal = get_ideal_expectation(bound_circuit, observable)

            row = {
                'circuit_id': current,
                'num_qubits': num_qubits,
                'num_layers': reps,
                'num_params': num_params,
                'x_ideal': x_ideal,
            }

            for noise_type in noise_types:
                for i, err in enumerate(error_rates):
                    col_name = f'x_{noise_type[:3]}_n{i}'
                    row[col_name] = get_noisy_expectation(
                        bound_circuit,
                        simulators[noise_type][i],
                        observable,
                        shots=2048
                    )

            data.append(row)
            current += 1

            if current % 500 == 0:
                elapsed = time.time() - start_time
                rate = current / elapsed
                remaining = (total_circuits - current) / rate
                print(f"  Progress: {current:,}/{total_circuits:,} ({100*current/total_circuits:.1f}%) "
                      f"| Remaining: {remaining/60:.1f}min")


df = pd.DataFrame(data)
df.to_csv('qem_database_large.csv', index=False)

print("\n" + "="*70)
print("DATABASE COMPLETE")
print("="*70)
print(f"Total circuits: {len(df):,}")
print(f"Saved to: qem_database_large.csv")


QEM DATABASE GENERATOR (FIXED)
Total circuits: 8,000

Processing 2 qubits...
  Progress: 500/8,000 (6.2%) | Remaining: 107.6min
  Progress: 1,000/8,000 (12.5%) | Remaining: 103.5min
  Progress: 1,500/8,000 (18.8%) | Remaining: 97.9min
  Progress: 2,000/8,000 (25.0%) | Remaining: 91.4min

Processing 3 qubits...
  Progress: 2,500/8,000 (31.2%) | Remaining: 99.3min
  Progress: 3,000/8,000 (37.5%) | Remaining: 100.2min
  Progress: 3,500/8,000 (43.8%) | Remaining: 97.0min
  Progress: 4,000/8,000 (50.0%) | Remaining: 91.2min

Processing 4 qubits...
  Progress: 4,500/8,000 (56.2%) | Remaining: 88.4min
